In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [55]:
df = pd.read_csv('./data/50_Startups.csv')

In [56]:
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [57]:
y = df['Profit']

In [58]:
X = df.drop('Profit',axis=1)

In [59]:
X.describe()

,R&D Spend,Administration,Marketing Spend
count,50.000000,50.000000,50.000000
mean,73721.615600,121344.639600,211025.097800
std,45902.256482,28017.802755,122290.310726
min,0.000000,51283.140000,0.000000
25%,39936.370000,103730.875000,129300.132500
50%,73051.080000,122699.795000,212716.240000
75%,101602.800000,144842.180000,299469.085000
max,165349.200000,182645.560000,471784.100000


In [60]:
sorted(df['State'].unique())

['California', 'Florida', 'New York']

In [61]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder

In [62]:
from sklearn.compose import ColumnTransformer

In [63]:
ct = ColumnTransformer([('onehot', OneHotEncoder(), [3])], remainder='passthrough')

In [64]:
#### 데이터 프리프로세싱 단계에서, 문자열을 숫자로 바꾸는 작업, 피쳐스케일링 등을 하면, 파이프라인으로 처리하는게 좋다.

In [65]:
print(X_train.dtypes)
print(y_train.dtypes)

R&D Spend          float64
Administration     float64
Marketing Spend    float64
State               object
dtype: object
R&D Spend          float64
Administration     float64
Marketing Spend    float64
State               object
dtype: object


In [66]:
from sklearn.linear_model import LinearRegression

In [67]:
regressor = LinearRegression()

In [68]:
from sklearn.pipeline import Pipeline

In [69]:
pipe = Pipeline(steps=[('preprocessing', ct), ('modeling', regressor)])

In [70]:
# 학습 / 데이터 준비
from sklearn.model_selection import train_test_split

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 20)

In [74]:
# 파이프라인 학습
pipe.fit(X_train, y_train)

,steps,"[('preprocessing', ...), ('modeling', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehot', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [75]:
y_pred = pipe.predict(X_test)

In [76]:
from sklearn.metrics import mean_squared_error, r2_score

In [77]:
mean_squared_error(y_test, y_pred)

103140891.12176925

In [78]:
r2_score(y_test, y_pred)

0.9438509847888678

In [79]:
import os
os.mkdir('./model')

In [80]:
import joblib

In [81]:
joblib.dump(pipe, './model/pipe.pkl')

['./model/pipe.pkl']

In [82]:
from sklearn.ensemble import RandomForestRegressor

In [83]:
regressor2 = RandomForestRegressor(random_state = 20)

In [85]:
pipe2 = Pipeline(steps = [('preprocessing', ct), ('modeling', regressor2)])

In [86]:
pipe2.fit(X_train, y_train)

,steps,"[('preprocessing', ...), ('modeling', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehot', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [87]:
y_pred2 = pipe2.predict(X_test)

In [88]:
mean_squared_error(y_test, y_pred2)

156402616.44411433

In [89]:
r2_score(y_test, y_pred2)

0.9148557590081949

In [90]:
from xgboost import XGBRegressor

In [91]:
regressor3 = XGBRegressor(random_state = 20)

In [92]:
pipe3 = Pipeline(steps=[('preprocessing', ct), ('modeling', regressor3)])

In [93]:
pipe3.fit(X_train, y_train)

,steps,"[('preprocessing', ...), ('modeling', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehot', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [94]:
y_pred3 = pipe3.predict(X_test)

In [95]:
mean_squared_error(y_test, y_pred3)

300487368.955356

In [96]:
r2_score(y_test, y_pred3)

0.836417257338721

In [97]:
from sklearn.model_selection import GridSearchCV

# 탐색할 하이퍼파라미터 후보 정의
param_grid = {
    "modeling__n_estimators": [100, 200, 300],
    "modeling__max_depth": [3, 5, 7],
    "modeling__learning_rate": [0.01, 0.05, 0.1],
    "modeling__subsample": [0.7, 0.9, 1.0],
    "modeling__colsample_bytree": [0.7, 0.9, 1.0]
}

# GridSearchCV 세팅
grid_search = GridSearchCV(
    estimator=pipe3,
    param_grid=param_grid,
    scoring="neg_mean_squared_error",  # 필요에 따라 R2, MAE 등 변경 가능
    cv=5,  # 5-fold 교차검증
    n_jobs=-1,
    verbose=2
)

# 학습
grid_search.fit(X_train, y_train)

# 최적 파라미터와 성능 출력
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score:", grid_search.best_score_)

# 테스트 세트에서 최종 성능 확인
y_pred = grid_search.predict(X_test)
from sklearn.metrics import mean_squared_error, r2_score
print("Test MSE:", mean_squared_error(y_test, y_pred))
print("Test R2:", r2_score(y_test, y_pred))

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best Parameters: {'modeling__colsample_bytree': 0.9, 'modeling__learning_rate': 0.1, 'modeling__max_depth': 5, 'modeling__n_estimators': 100, 'modeling__subsample': 0.7}
Best CV Score: -108511865.51778774
Test MSE: 118175472.74599274
Test R2: 0.9356662876902662
